In [1]:
import pandas as pd
import pickle
import numpy as np
import torch
from torch import nn
import os 
import fastai
import fastai
from fastai.text import *
PATH="/home/kirana/Documents/phd/final"
DATAPATH="/home/kirana/Documents/phd/data/aclImdb"

## Dataset

In [2]:
[df_train,df_valid,df_test,itos, train_tokens, valid_tokens, test_tokens, trn_lm, val_lm, test_lm]=pickle.load(open(f'{PATH}/inter/dfs_tokens_fastai.pkl','rb'))

In [3]:
bs=52 # 52 - Jeremey, 20 - default

In [4]:
bptt= 70 #70 - Jeremey, 35 - default

In [5]:
def create_data_lm(tokens,bs):
    import itertools
    # Collapse into a single large array
    tokens=np.asarray(list (itertools.chain(*tokens)))
    # How many batches
    n_batch=len(tokens)//bs
    # Truncate to exclude the ones at the end
    tokens=tokens[:bs*n_batch]
    # Reshape
    tokens=tokens.reshape(bs,-1)
    return tokens

In [6]:
train_tokens=create_data_lm(df_train['tokens'],bs)
valid_tokens=create_data_lm(df_valid['tokens'],bs)
test_tokens=create_data_lm(df_test['tokens'],bs)
train_tokens.shape, valid_tokens.shape, test_tokens.shape

((52, 396650), (52, 43895), (52, 143259))

In [7]:
def get_batch(source, i):
    seq_len = min(args.bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].view(-1)
    return data, target

In [8]:
n_batch=train_tokens.shape[1]
for i in range(0,n_batch,bptt):
    seq_len=min(bptt,n_batch-1-i)
    x=train_tokens[:,i:i+seq_len]
    y=train_tokens[:,i+1:i+1+seq_len]
    print (x.shape, y.shape)
    break

(52, 70) (52, 70)


In [9]:
x.shape,y.shape

((52, 70), (52, 70))

In [10]:
x,y

(array([[   41,    42,    43,    40, ...,     2,  1896,   463,     4],
        [ 1071,    23,    67,    39, ...,     2,    98,    11,    63],
        [   54,    34,     7,   190, ...,     2,    87,    34,  5998],
        [    7,  1213,     8,   453, ...,   406,     4,     2,    46],
        ...,
        [   60,   126,   931,     6, ...,   114,    39,   599,    63],
        [    2, 15094,   132,    65, ...,    56,  1350,   209,   281],
        [11783,   658,   318,     3, ...,     6,    64,    79,  1268],
        [   42,    43,    40,     2, ...,   708,   111,     4,     2]]),
 array([[   42,    43,    40,    13, ...,  1896,   463,     4,     2],
        [   23,    67,    39,    18, ...,    98,    11,    63,    28],
        [   34,     7,   190,  1732, ...,    87,    34,  5998,    47],
        [ 1213,     8,   453,    21, ...,     4,     2,    46,     7],
        ...,
        [  126,   931,     6,   126, ...,    39,   599,    63,   314],
        [15094,   132,    65,   411, ...,  1350, 

In [11]:
x.shape,y.shape

((52, 70), (52, 70))

In [12]:
train_tokens.shape

(52, 396650)

## pre-trained GLOVE

In [13]:
PATH_GLOVE='/home/kirana/Documents/phd/data/pre-trained/glove/glove.6B'

In [14]:
ls {PATH_GLOVE}

glove_50_glove_100.tgz  glove.6B.300d.txt  word2idx_glove50
glove.6B.100d.txt       glove.6B.50d.txt   wordvectors_glove200
glove.6B.200d.txt       word2idx_glove200  wordvectors_glove50


In [15]:
def load_glove_vectors(path=f'{PATH_GLOVE}/glove.6B.50d.txt'):
    f=open(f'{path}','r')
    glovelines=f.readlines()
    word2idx={}
    idx=0
    wordvectors={}
    for line in glovelines:
        line=line.split(' ')
        vector=np.asarray(line[1:]).astype(np.float32)
        word2idx[line[0]]=idx
        idx=idx+1
        wordvectors[line[0]]=vector
    return word2idx,wordvectors

In [16]:
word2idx_glove,wordvectors_glove=load_glove_vectors(f'{PATH_GLOVE}/glove.6B.300d.txt')
import pickle
pickle.dump(word2idx_glove,open(f'{PATH_GLOVE}/word2idx_glove300','wb'))
pickle.dump(wordvectors_glove,open(f'{PATH_GLOVE}/wordvectors_glove300','wb'))

In [19]:
word2idx_glove=pickle.load(open(f'{PATH_GLOVE}/word2idx_glove300','rb'))
wordvectors_glove=pickle.load(open(f'{PATH_GLOVE}/wordvectors_glove300','rb'))

In [17]:
len(word2idx_glove)

400000

In [18]:
[*word2idx_glove.keys()][:5]

['the', ',', '.', 'of', 'to']

In [19]:
wordvectors_glove['the'].shape

(300,)

In [20]:
glove_mtx=np.array([wordvectors_glove[key] for key in [*word2idx_glove.keys()]])

In [21]:
glove_mtx.shape

(400000, 300)

In [22]:
# In our list but not in Glove
len(set(itos).difference(set([*word2idx_glove.keys()])))

5072

In [23]:
len(set(itos).difference(set([*word2idx_glove.keys()])))/len(itos)

0.0845728006402988

In [24]:
glove_mtx=np.zeros((len(itos),glove_mtx.shape[1]))

In [25]:
wordvectors_glove['the']

array([ 0.04656 ,  0.21318 , -0.007436, -0.45854 , ..., -0.087461,  0.009061, -0.20989 ,  0.053913], dtype=float32)

In [26]:
enc_wgts = glove_mtx # converts np.ndarray from torch.FloatTensor.output shape: (238462, 400)
row_m = enc_wgts.mean(0) # returns the average of the array elements along axis 0. output shape: (400,)

In [27]:
for i,word in enumerate(itos):
    try:
        glove_mtx[i]=wordvectors_glove[word]
    except:
        glove_mtx[i]=row_m

In [28]:
glove_mtx[3]

array([ 0.04656 ,  0.21318 , -0.007436, -0.45854 , ..., -0.087461,  0.009061, -0.20989 ,  0.053913])

In [29]:
enc_wgts.shape, len(itos)

((59972, 300), 59972)

In [30]:
n_emb=enc_wgts.shape[1]

In [31]:
new_w=enc_wgts

In [32]:
new_w.shape

(59972, 300)

## Model Architecture

In [33]:
from torch import nn
import torch
from adaptive import *
from torch.autograd import Variable

In [35]:
n_inp=len(itos)
n_emb=300 #650
n_hidden=300 #650
n_layers=2
dropout=0.5
wd=1e-7

In [36]:
class language_model (nn.Module):
    def __init__(self,n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e=0.05,dropout=0.5,\
                 dropout_o=0.5,pretrain_mtx=None,adaptive_log_softmax=True,tie_weights=True):
        super().__init__()
        self.n_inp,self.n_emb,self.n_hidden,self.n_layers,self.bidirectional,self.bs,self.device,self.pretrain_mtx=\
                            n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,pretrain_mtx
        self.adaptive_log_softmax,self.tie_weights=adaptive_log_softmax,tie_weights
        self.dropout_e,self.dropout,self.dropout_o=dropout_e,dropout,dropout_o
        self.gen_hidden()
        self.create_architecture()
        if pretrain_mtx is not None:
            print ("initializing")
            self.initialize_glove()
            
        if self.adaptive_log_softmax is False:
            self.criterion=nn.CrossEntropyLoss()
        
    def create_architecture(self):
        # Dropout layer
        self.dropout_enc=nn.Dropout(self.dropout_e)
        # Embedding Layer
        self.encoder=nn.Embedding(self.n_inp,self.n_emb)
        # LSTM Layer
        self.lstm=nn.LSTM(self.n_emb,self.n_hidden,self.n_layers,batch_first=True,dropout=self.dropout,\
                          bidirectional=False)
        self.dropout_op=nn.Dropout(self.dropout_o)
        
        if self.adaptive_log_softmax:
            # Adaptive Log Softmax Loss
            self.adaptive_softmax=AdaptiveLogSoftmaxWithLoss(self.n_hidden,
                                    self.n_inp,
                                    cutoffs=[round(self.n_inp/15),3*round(self.n_inp/15)],
                                    div_value=4,
                                    get_full_prob=True)
        else:
            self.decoder=nn.Linear(self.n_hidden,self.n_inp)
    
    def freeze_embedding(self):
        self.encoder.weight.requires_grad=False
        if self.tie_weights:
            self.decoder.weight.requires_grad=False
        
    
    def unfreeze_embedding(self):
        self.encoder.weight.requires_grad=True
        if self.tie_weights:
            self.decoder.weight.requires_grad=True
        
    def initialize_glove(self):
        self.encoder.weight.data=torch.Tensor(self.pretrain_mtx)
        if self.tie_weights:
            self.decoder.weight=self.encoder.weight
    
    def gen_hidden(self):
        # Initialize hidden
        self.hidden=(Variable(torch.zeros(self.n_layers,self.bs,self.n_hidden,requires_grad=False).to(self.device)),
                     Variable(torch.zeros(self.n_layers,self.bs,self.n_hidden,requires_grad=False).to(self.device)))
    
        
    def forward(self,Xb,Yb):
        #import pdb
        #pdb.set_trace()
        embs=self.dropout_enc(self.encoder(Xb))
        if Xb.size(0) < self.bs:
            self.hidden=(self.hidden[0][:,:Xb.size(0),:].contiguous(),
            self.hidden[1][:,:Xb.size(0),:].contiguous())
        out,new_hidden=self.lstm(embs,self.hidden)
        out=self.dropout_op(out)
         # Wrap the hidden state in a new tensor without the gradients
        self.hidden=(Variable(new_hidden[0].data,requires_grad=False).to(self.device),\
                     Variable(new_hidden[1].data,requires_grad=False).to(self.device))
        if self.adaptive_log_softmax:
            out=out.reshape(out.size(0)*out.size(1),out.size(2))        # output is of shape n_batch * n_seq * n_hidden
      
            out=self.adaptive_softmax(out,Yb.view(-1))
            loss=out.loss
            preds=out.output_full
        else:
            #import pdb
            #pdb.set_trace()
            preds=self.decoder(out.contiguous().view(out.size(0)*out.size(1), out.size(2)))
            loss=self.criterion(preds,Yb.contiguous().view(-1))

        return preds, loss  

In [37]:
def accuracy_multinomial(preds,actual):
    preds=preds.max(1)[1]
    correct=preds==actual
    return correct.float().sum()/len(correct)

In [38]:
device="cuda:0"
model=language_model(n_inp,n_emb,n_hidden,n_layers,False,bs,device,0.05,0.5,0.5,new_w,False,True)


initializing


In [39]:
new_w.shape

(59972, 300)

In [40]:
model.encoder.weight.data.shape,model.decoder.weight.data.shape

(torch.Size([59972, 300]), torch.Size([59972, 300]))

In [41]:
nn.Linear(n_hidden,n_inp).weight.data.shape, n_hidden, n_inp

(torch.Size([59972, 300]), 300, 59972)

### Test if model forward works

In [42]:
model.decoder.weight.shape,model.encoder.weight.shape

(torch.Size([59972, 300]), torch.Size([59972, 300]))

In [43]:
x.shape

(52, 70)

In [44]:
if 1==0:
    model.forward(torch.LongTensor(x),torch.LongTensor(y))

In [45]:

model=model.to(device)

## Learner

In [46]:
class Learner():
    def __init__(self,model,optimizer,metric_fn,device,bptt=12,print_every=5,clip_val=None):
        self.model,self.optimizer,self.metric_fn,self.device,self.print_every,self.bptt,self.losses,self.clip_val=\
            model,optimizer,metric_fn,device,print_every,bptt,[],clip_val
        self.n_epochs=1
  
        
    
    def fit (self,Xb,Yb,mode_train=True):
        if mode_train:
            self.model.train()
        else:
            self.model.eval()
            
        preds,loss=self.model(Xb,Yb)
        
       
            
        
        with torch.no_grad():
            acc=self.metric_fn(preds,Yb.view(-1))
            acc=acc.item()
            del preds
        
        if mode_train:
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
        myloss=loss.item()
        del loss
        
        if self.clip_val is not None:
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), 0.25)
            if 1==0:
                for p in self.model.parameters():
                    p.data.add_(self.lr, p.grad.data)
        
        return myloss, acc
    
    def lr_find (self,start_lr,end_lr,iterator,n_batch):
        losses,lrs=[],[]
        ratio=end_lr/start_lr
        num_steps=n_batch
        lr=start_lr
        for i in range(num_steps):            
            lr=lr*(end_lr/start_lr)**(1/num_steps)
            lrs.append(lr)
        self.lrs=lrs
        self.run_epoch(iterator,mode_train=True,lrs=lrs)
    
    def run_epoch(self,iterator,mode_train,lrs=None):
        n_batch=iterator.shape[1]
        epoch_loss,epoch_acc,i=0,0,0
        self.model.gen_hidden()
        for k,i in enumerate(range(0,n_batch,self.bptt)):
            seq_len=min(bptt,n_batch-1-i)
            Xb=train_tokens[:,i:i+seq_len]
            Yb=train_tokens[:,i+1:i+1+seq_len]
            Xb=torch.LongTensor(Xb)
            Yb=torch.LongTensor(Yb)
            Xb=Xb.to(self.device)
            Yb=Yb.to(self.device)
            
            if lrs is not None:
                lr=lrs[k]
                for param_group in self.optimizer.param_groups:
                    param_group['lr']=lr 
            

            loss,acc=self.fit(Xb,Yb,mode_train)
            
            if lrs is not None:
                self.losses.append(loss)
            
            
            epoch_loss+=loss
            epoch_acc+=acc
            if k%self.print_every == 0:
                if k:
                    print (f'Batch:{k} {epoch_loss/(k)}  {epoch_acc/(k)}')  
                    torch.cuda.empty_cache()
        epoch_loss=epoch_loss/k
        epoch_acc=epoch_acc/k
        
        if 1==0:
            lr /= 4.0
            # Freeze all the layers initially
            for param in resnet.parameters():
                param.requires_grad=False
            torch.save(resnet,'resnet')
            torch.save(resnet.state_dict(),'resnet_state_dict')
            resnet.load_state_dict(torch.load('resnet_state_dict'))
            for param_group in self.optimizer.param_groups:
                    param_group['lr']=lr  
            
        return epoch_loss,epoch_acc
    
    def plot_lrs(self, n_roll=1):
        import seaborn as sns
        ax=sns.lineplot(x=self.lrs,y=pd.Series(self.losses).rolling(n_roll).mean())
        ax.set_xscale('log')
        ax.set_ylabel('Loss')
        ax.set_xlabel('Learning Rate')

    
    def run_epochs(self,dltrain,dlvalid,n_epochs=1):
        for epoch in range(n_epochs):
            
            loss,acc=self.run_epoch(dltrain,True)
            print (f'Epoch:{epoch} Loss:{loss}')
            lossv,accv=self.run_epoch(dlvalid,mode_train=False)
            print (f'Epoch:{epoch} Loss:{loss} Accuracy:{acc} Loss:{lossv} Accuracy:{accv}')
        

In [47]:
optimizer=torch.optim.Adam(model.parameters(),lr=1e-3,betas=(0.9,0.999), weight_decay=wd)
metric_fn=accuracy_multinomial

In [48]:
n_batch=np.int(np.ceil(train_tokens.shape[1]/bptt))
n_batch

5667

In [49]:
model.freeze_embedding()

In [50]:
model.encoder.weight.requires_grad, model.decoder.weight.requires_grad

(False, False)

In [51]:
learner=Learner(model,optimizer,accuracy_multinomial,device,bptt,500,0.25)

In [52]:
model.n_inp

59972

In [53]:
learner.run_epochs(train_tokens,valid_tokens,3)

Batch:500 7.650373255729675  0.0643522001367528
Batch:1000 7.355331929206848  0.08340714605210815
Batch:1500 7.16919517103831  0.09369890569189253
Batch:2000 7.026159649848938  0.10086923582880991
Batch:2500 6.907291049194336  0.10625385119938291
Batch:3000 6.804987421353658  0.11059451041638385
Batch:3500 6.713873579570225  0.11428846648352088
Batch:4000 6.631647217273712  0.11763661232418962
Batch:4500 6.557406237072414  0.12086105564084007
Batch:5000 6.489825654697418  0.12414945648175199
Batch:5500 6.426086421272972  0.12767707912905396
Epoch:0 Loss:6.406246617452188
Batch:500 5.427806157112122  0.19353791946172713
Epoch:0 Loss:6.406246617452188 Accuracy:0.12880557513698568 Loss:5.425855957530142 Accuracy:0.19330056650977007
Batch:500 5.706937162399292  0.17305165627598762
Batch:1000 5.671671550750732  0.17569011788070202
Batch:1500 5.645417999903361  0.1780686892469724
Batch:2000 5.619542998313904  0.18057445825636387
Batch:2500 5.5962138168334965  0.1825322054207325
Batch:3000 5.

In [54]:
learner.run_epochs(train_tokens,valid_tokens,2)

Batch:500 5.362048350334168  0.20057363364100456
Batch:1000 5.356271614551544  0.20035824891924858
Batch:1500 5.3562648229599  0.20016923805077871
Batch:2000 5.354249463319778  0.20030934796482325
Batch:2500 5.352171015357971  0.2003825347840786
Batch:3000 5.350640781402588  0.20040843222041924
Batch:3500 5.348602713312421  0.20052802929282187
Batch:4000 5.346792228221894  0.20064732870459556
Batch:4500 5.345160547150506  0.20077168007029428
Batch:5000 5.344375391769409  0.20087138094305992
Batch:5500 5.342533753828569  0.20105749984762886
Epoch:0 Loss:5.342134636533248
Batch:500 5.02782037448883  0.22905990239977836
Epoch:0 Loss:5.342134636533248 Accuracy:0.20107552953303764 Loss:5.026465336101477 Accuracy:0.22885690092375024
Batch:500 5.334267107963562  0.20334726029634476
Batch:1000 5.3293809261322025  0.20306566692888736
Batch:1500 5.329662696520487  0.20273114311695098
Batch:2000 5.328221499919891  0.20278723277151584
Batch:2500 5.326817391586304  0.20271758991479874
Batch:3000 5.

In [55]:
torch.save(model.state_dict(),f'{PATH}/inter/glove300_model_state_dict')
torch.save(optimizer.state_dict(),f'{PATH}/inter/glove300_learner_state_dict')

In [56]:
model.unfreeze_embedding()

In [57]:
model.encoder.weight.requires_grad, model.decoder.weight.requires_grad

(True, True)

In [58]:
learner.run_epochs(train_tokens,valid_tokens,3)

Batch:500 5.109595203399659  0.22029616543650626
Batch:1000 5.024958127498627  0.22427748450636864
Batch:1500 4.976035773277283  0.22652272260189057
Batch:2000 4.937592483520508  0.2284240504875779
Batch:2500 4.90689591217041  0.22983078132867812
Batch:3000 4.881629475434621  0.2310493710587422
Batch:3500 4.8595305418287005  0.23218172320723535
Batch:4000 4.840361280798912  0.23312686644122005
Batch:4500 4.823488499323527  0.23402217317620913
Batch:5000 4.808894626903534  0.23476704494059086
Batch:5500 4.794439581437544  0.23558677515387536
Epoch:0 Loss:4.790255012505474
Batch:500 4.369141964912415  0.26816264814138413
Epoch:0 Loss:4.790255012505474 Accuracy:0.23581459306172586 Loss:4.368037181227591 Accuracy:0.26794916671809205
Batch:500 4.637830147743225  0.24487913247942925
Batch:1000 4.627820488929749  0.24505742911994458
Batch:1500 4.624270217259725  0.24494964515169462
Batch:2000 4.618249243497848  0.24523284111917018
Batch:2500 4.612601367759704  0.24534979175329208
Batch:3000 4

In [59]:
torch.save(model.state_dict(),f'{PATH}/inter/glove300_model_state_dict')
torch.save(optimizer.state_dict(),f'{PATH}/inter/glove300_learner_state_dict')

In [60]:
learner.run_epochs(train_tokens,valid_tokens,3)

Batch:500 4.4805640687942505  0.25539451655745504
Batch:1000 4.474158131122589  0.2550252858698368
Batch:1500 4.473895053227743  0.2547846265236537
Batch:2000 4.471721058607101  0.25485619227588174
Batch:2500 4.469276383209229  0.25479858236908914
Batch:3000 4.46733182032903  0.254842776487271
Batch:3500 4.465054332051959  0.2549701836151736
Batch:4000 4.462561264872551  0.2551188982762396
Batch:4500 4.460545792579651  0.2552862135635482
Batch:5000 4.459197494506836  0.2553994614362717
Batch:5500 4.456690646691756  0.25562358725883744
Epoch:0 Loss:4.456210838288247
Batch:500 4.1550737524032595  0.28319616737961767
Epoch:0 Loss:4.456210838288247 Accuracy:0.2556529957110748 Loss:4.154098665124872 Accuracy:0.28289825560753806
Batch:500 4.441973182678223  0.2583093514740467
Batch:1000 4.43620744562149  0.2580338017642498
Batch:1500 4.436528824806214  0.25765349028507867
Batch:2000 4.434580546855926  0.25761292251199486
Batch:2500 4.432978987693787  0.25742198864817617
Batch:3000 4.43162754

In [62]:
torch.save(model.state_dict(),f'{PATH}/inter/glove50_model_state_dict')
torch.save(optimizer.state_dict(),f'{PATH}/inter/glove50_learner_state_dict')

In [63]:
learner.run_epochs(train_tokens,valid_tokens,2)

Batch:500 4.394060876846313  0.2616000111401081
Batch:1000 4.3888170924186705  0.2612609998434782
Batch:1500 4.3895150696436565  0.2608487287759781
Batch:2000 4.388436628103256  0.260814021833241
Batch:2500 4.386969807815552  0.2607619886636734
Batch:3000 4.38608130645752  0.26082647589345775
Batch:3500 4.384810282434736  0.2608701048578535
Batch:4000 4.383412978053093  0.26090749687701464
Batch:4500 4.382347243203057  0.2609957371254762
Batch:5000 4.3818060770988465  0.26106951603591444
Batch:5500 4.380182161331176  0.26122533525391056
Epoch:0 Loss:4.3799635115925435
Batch:500 4.081819926738739  0.28890056455135343
Epoch:0 Loss:4.3799635115925435 Accuracy:0.2612428658270726 Loss:4.081071397714448 Accuracy:0.2885811706526618
Batch:500 4.376593412399292  0.26288462603092194
Batch:1000 4.371794269084931  0.2627299554347992
Batch:1500 4.373024185498555  0.262359717498223
Batch:2000 4.3721199862957  0.26224218074977396
Batch:2500 4.370902025222779  0.26204902150034903
Batch:3000 4.37020948

In [64]:
np.exp(3.9), np.exp(4.09)

(49.40244910553017, 59.739891704145194)

In [65]:
if 1==0:
    learner.lr_find(1e-4,1e-1,valid_tokens,n_batch)
    learner.plot_lrs(50)
    len(learner.lrs)
    len(learner.losses)

In [66]:
torch.save(model.state_dict(),f'{PATH}/inter/glove300_model_state_dict')
torch.save(optimizer.state_dict(),f'{PATH}/inter/glove300_learner_state_dict')

## Transfer Learning to predict RNN